In [ ]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import soupsieve as sv

#### initialize empty lists

In [ ]:
sites_list = []
namelist = []
induct_year = []
clean_induct_year = []

#### create sites_list dynamically

In [ ]:
firstpage = "https://countrymusichalloffame.org/hall-of-fame/members/"
url_base = "https://countrymusichalloffame.org/hall-of-fame/members/page/" #no numbers here plz
startpage = 2
lastpage = 17 # make sure to go one more for python reasons!

sites_list.append(firstpage)

for i in range(startpage, lastpage):
    sites_list.append(url_base + str(i))

#### loop through list

In [ ]:
for site in sites_list:
    response = requests.get(site)
    soup = BeautifulSoup(response.content, 'lxml')
    # loop to get names
    for names in soup.find_all('h3'):
        namelist.append(names.get_text())
    # loop to get induction years
    my_regex = re.compile(r"Inducted")
    for year in soup.find_all(text = my_regex):
        induct_year.append(year)

In [ ]:
#cleaning induct_year output
for el in induct_year:
    clean_induct_year.append(el[-22:-18])

In [ ]:
### another way to get text from h3 tags
# artist_list = [artist.text for artist in artist_names]

### convert lists to df

In [ ]:
country_df = pd.DataFrame()
country_df['names'] = namelist
country_df['Induction Year'] = clean_induct_year
country_df[-20:]

### fixing datatypes of columns

In [ ]:
data_types_dict = {'names': str, 'Induction Year': 'int64'}
country_df = country_df.astype(data_types_dict)
country_df.info()

In [ ]:
# country_df.value_counts("Induction Year") 
# country_df.sort_values("Induction Year")
# plt.bar(country_df["Induction Year"], country_df["Induction Year"].value_counts())

country_df['Induction Year'].value_counts().sort_index().plot(kind='bar', figsize=(18, 6));


In [ ]:
country_df['namelen'] = country_df.apply(lambda x: len(x.names), axis=1)
country_df[0:50]

In [ ]:
sorted_df = country_df.sort_values("Induction Year")
sorted_df


In [ ]:
sns.stripplot(y = sorted_df["Induction Year"], x = sorted_df['namelen'])
plt.title("Name lengths of Winners Over Decades")
plt.show()

### Bonus Zone

In [ ]:
#list for all p tags with info we want
biolinks = []

In [ ]:
#CHEATING!! functions I wrote before with soup sieve
def get_attr(sv_target, targetlist, soup, desired_attr="href"):
    key_tags1 = sv.select(sv_target, soup)
    for link in key_tags1:
        targetlist.append(link.get(desired_attr))

def get_text(sv_target, targetlist, soup):
    key_tags2 = sv.select(sv_target, soup)
    for tag2 in key_tags2:
        targetlist.append(tag2.text)

In [ ]:
for site in sites_list:
    response = requests.get(site)
    soup = BeautifulSoup(response.content, 'lxml')
    smallsoup = soup.find('div', {'class': 'vertical-cards-container'})
    #loop to get bio urls
    get_attr("div.fbyf-btn-wrapper.inline-text > a", biolinks, smallsoup)

In [ ]:
len(biolinks)

### getting info on each bio page

In [ ]:
lifeinfo = []

In [ ]:
for site in biolinks:
    response = requests.get(site)
    soup = BeautifulSoup(response.content, 'lxml')
    targetsoup = soup.find('div', {'class': 'hero-left'})
    get_text("div p:last-child", lifeinfo, targetsoup)

In [ ]:
len(lifeinfo)

### pulling out specfic infos we need

In [ ]:
birthlist = []
deathlist = []
birthplacelist = []

In [ ]:
for line in lifeinfo:
    try:
        birthdate = re.search(r'Birth:\s\w+\s\d+,\s\d{4}', line)
        birthlist.append(birthdate.group())
    except:
        birthlist.append("")
    try:
        deathdate = re.search(r'Death:\s\w+\s\d+,\s\d{4}', line)
        deathlist.append(deathdate.group())
    except:
        deathlist.append("")
    try:
        birthplace = re.search(r'Birthplace:\s\w+,\s\D+', line) #this needs fixin!
        birthplacelist.append(birthplace.group())
    except:
        birthplacelist.append("")
        

### cleaning up the lists a bit

In [ ]:
cleanbirth = []
cleandeath = []
cleanbirthplace = []

In [ ]:
# birthlist[0]

In [ ]:
# birthlist[0][7:]

In [ ]:
for line in birthlist:
    cleanline = line[7:]
    cleanbirth.append(cleanline)

In [ ]:
# cleanbirth

In [ ]:
# deathlist

In [ ]:
for line in deathlist:
    cleanline = line[7:]
    cleandeath.append(cleanline)

In [ ]:
# cleandeath

In [ ]:
# birthplacelist

In [ ]:
# birthplacelist[0][12:]

In [ ]:
for line in birthplacelist:
    cleanline = line[12:]
    cleanbirthplace.append(cleanline)

In [ ]:
# cleanbirthplace

### add clean lists to country_df

In [ ]:
# country_df = country_df.drop("Birth", axis = 1)

In [ ]:
country_df["Birth Date"] = cleanbirth
country_df["Death Date"] = cleandeath
country_df["Birthplace"] = cleanbirthplace

# final result

In [ ]:
country_df

### cool ideas for this data:
- visualize how old people were when they were inducted (avg. age, median age, max age, min age, etc.)
- see how many people were inducted before vs. after their death (would need to convert columns to datetime for comparison)